In [ ]:
import numpy as np
from GDTR import GeneticDecisionTreeRegressor
from or_gym.envs.supply_chain.inventory_management import InvManagementMasterEnv
import random
from scipy.optimize import minimize
%load_ext autoreload
%autoreload 2

### Functions

In [ ]:
def _update_inventory_state(env):
        '''
        Get current state of the system: Inventory position at each echelon
        Inventory at hand + Pipeline inventory - backlog up to the current stage 
        (excludes last stage since no inventory there, nor replenishment orders placed there).
        '''
        n = env.period
        m = env.num_stages
        #print(env.I[n, :])
        if n>=1:
            IP = np.cumsum(env.I[n,:] + env.T[n,:] - env.B[n-1,:-1])
            #print(IP)
        else:
            IP = np.cumsum(env.I[n,:] + env.T[n,:])
            #print(IP)
        env.state = IP
        return env.state
        
        
def base_stock_action(env ,z):
        '''
        Sample action (number of units to request) based on a base-stock policy (order up to z policy)
        z = [integer list; dimension |Stages| - 1] base stock level (no inventory at the last stage)
        '''
        n = env.period
        c = env.supply_capacity
        m = env.num_stages
        IP = _update_inventory_state(env) # extract inventory position (current state)
        
        try:
            dimz = len(z)
        except:
            dimz = 1
        assert dimz == m-1, "Wrong dimension on base stock level vector. Should be # Stages - 1."
        
        #print(type(IP))
        #print(IP)
        # calculate total inventory position at the beginning of period n
        R = z - IP # replenishmet order to reach zopt

        # check if R can actually be fulfilled (capacity and inventory constraints)
        Im1 = np.append(env.I[n,1:], np.Inf) # available inventory at the m+1 stage
                                            # NOTE: last stage has unlimited raw materials
        Rpos = np.column_stack((np.zeros(len(R)),R)) # augmented materix to get replenishment only if positive
        A = np.column_stack((c, np.max(Rpos,axis=1), Im1)) # augmented matrix with c, R, and I_m+1 as columns
        
        R = np.min(A, axis = 1) # replenishmet order to reach zopt (capacity constrained)
        
        return R


def min_max_action(env, z_min, z_max): # (s, S) policy
    '''
    Sample action (number of units to request) based on a min-max policy (order up to z_max if inv is below z_min)
    z = [integer list; dimension |Stages| - 1] base stock level (no inventory at the last stage)
    '''
    assert len(z_min) == len(z_max)
    
    n = env.period
    c = env.supply_capacity
    m = env.num_stages
    IP = _update_inventory_state(env) # extract inventory position (current state)
    
    try:
        dimz = len(z_min)
    except:
        dimz = 1
    assert dimz == m-1, "Wrong dimension on base stock level vector. Should be # Stages - 1."
    
    R = np.zeros(3)
    #print(type(IP))
    #print(IP)
    # calculate total inventory position at the beginning of period n
    for i in range(len(IP)):
        if IP[i] < z_min[i]:
            R[i] = z_max[i] - IP[i] # replenishmet order to reach z_max
        else:
            R[i] = 0

    # check if R can actually be fulfilled (capacity and inventory constraints)
    Im1 = np.append(env.I[n,1:], np.Inf) # available inventory at the m+1 stage
                                        # NOTE: last stage has unlimited raw materials
    Rpos = np.column_stack((np.zeros(len(R)),R)) # augmented materix to get replenishment only if positive
    A = np.column_stack((c, np.max(Rpos,axis=1), Im1)) # augmented matrix with c, R, and I_m+1 as columns
    
    R = np.min(A, axis = 1) # replenishmet order to reach zopt (capacity constrained)
    
    return R


def R_S_action(env, R_period, S): # kinda like base-stock, but without the continuous review
    '''
    Inventory level is reviewed every R_period periods.
    Right after the review an order is placed bringing the inventory level to the predefined level S
    '''
    n = env.period
    
    if n % R_period != 0:
        R = np.zeros(3) # 3 being the 
        
    else:
        c = env.supply_capacity
        m = env.num_stages
        IP = _update_inventory_state(env) # extract inventory position (current state)
        
        try:
            dimz = len(S)
        except:
            dimz = 1
        assert dimz == m-1, "Wrong dimension on base stock level vector. Should be # Stages - 1."
        
        #print(type(IP))
        #print(IP)
        # calculate total inventory position at the beginning of period n
        R = S - IP # replenishmet order to reach zopt

        # check if R can actually be fulfilled (capacity and inventory constraints)
        Im1 = np.append(env.I[n,1:], np.Inf) # available inventory at the m+1 stage
                                            # NOTE: last stage has unlimited raw materials
        Rpos = np.column_stack((np.zeros(len(R)),R)) # augmented materix to get replenishment only if positive
        A = np.column_stack((c, np.max(Rpos,axis=1), Im1)) # augmented matrix with c, R, and I_m+1 as columns
        
        R = np.min(A, axis = 1) # replenishmet order to reach zopt (capacity constrained)
    
    return R


def R_s_S_action(env, R_period, s, S): # like min-max, but without the continuous review
    '''
    Inventory level is reviewed every R_period periods and as soon as it passes a reorder point s, 
    an order is placed bringing the inventory level to the predefined level S. 
    '''
    n = env.period
    
    if n % R_period:
        R = np.zeros(3)
    
    else:
        assert len(s) == len(S)
    
        c = env.supply_capacity
        m = env.num_stages
        IP = _update_inventory_state(env) # extract inventory position (current state)
        
        try:
            dimz = len(s)
        except:
            dimz = 1
        assert dimz == m-1, "Wrong dimension on base stock level vector. Should be # Stages - 1."
        
        R = np.zeros(3)
        #print(type(IP))
        #print(IP)
        # calculate total inventory position at the beginning of period n
        for i in range(len(IP)):
            if IP[i] < s[i]:
                R[i] = S[i] - IP[i] # replenishmet order to reach z_max
            else:
                R[i] = 0

        # check if R can actually be fulfilled (capacity and inventory constraints)
        Im1 = np.append(env.I[n,1:], np.Inf) # available inventory at the m+1 stage
                                            # NOTE: last stage has unlimited raw materials
        Rpos = np.column_stack((np.zeros(len(R)),R)) # augmented materix to get replenishment only if positive
        A = np.column_stack((c, np.max(Rpos,axis=1), Im1)) # augmented matrix with c, R, and I_m+1 as columns
        
        R = np.min(A, axis = 1) # replenishmet order to reach zopt (capacity constrained)
    
    return R


def r_Q_action(env, r, Q):
    '''
    Inventory level is reviewed continuously, 
    and as  soon as  inventory level reaches the threshold r, an order of size Q is placed.
    '''
    n = env.period
    c = env.supply_capacity
    m = env.num_stages
    IP = _update_inventory_state(env) # extract inventory position (current state)
    
    try:
        dimz = len(Q)
    except:
        dimz = 1
    assert dimz == m-1, "Wrong dimension on base stock level vector. Should be # Stages - 1."
    
    R = np.zeros(3)
    
    for i in range(len(IP)):
        if IP[i] < r[i]:
            R[i] = Q[i]
        else:
            R[i] = 0

    # check if R can actually be fulfilled (capacity and inventory constraints)
    Im1 = np.append(env.I[n,1:], np.Inf) # available inventory at the m+1 stage
                                        # NOTE: last stage has unlimited raw materials
    Rpos = np.column_stack((np.zeros(len(R)),R)) # augmented materix to get replenishment only if positive
    A = np.column_stack((c, np.max(Rpos,axis=1), Im1)) # augmented matrix with c, R, and I_m+1 as columns
    
    R = np.min(A, axis = 1) # replenishmet order to reach zopt (capacity constrained)
    
    return R



### Optimised Base-Stock Policy

In [ ]:
def base_stock_black_box(base_stock):
    env = InvManagementMasterEnv(periods=30, c=[4, 3, 2], I0=[3, 2, 2], dist_param={'mu': 1.2}, L=[1, 1, 1])

    N_episodes = 10 # NOTE: we chose this
    total_reward_dummy = np.array([])

    for i in range(N_episodes):
        obs = env.reset()
        done = False
        
        #obs, reward, done = env.sample_action()
        
        while not done:
            #print(env.I)
            action = base_stock_action(env, base_stock)

            # Take the defined action (place an order), and advance to the next time period by taking a "step"
            obs, reward, done, _ = env.step(action)
            total_reward_dummy = np.append(total_reward_dummy, [reward])
            
    avg_total_rewards = np.average(total_reward_dummy)
    
    return avg_total_rewards



n_restarts = 100
best_base_stock = -99999

for i in range(n_restarts):
    x0 = np.array([random.randint(0, 10) for i in range(3)])
    opt_base_stock = minimize(base_stock_black_box, x0)
    #print(opt_base_stock.fun)
    
    if opt_base_stock.fun > best_base_stock:
        best_base_stock = opt_base_stock.fun

best_base_stock

### Optimised Min-Max Policy

In [ ]:
def min_max_black_box(bounds):
    z_min = bounds[:int(len(bounds)/2)]
    z_max = bounds[int(len(bounds)/2):]
    
    env = InvManagementMasterEnv(periods=30, c=[4, 3, 2], I0=[3, 2, 2], dist_param={'mu': 1.2}, L=[1, 1, 1])

    N_episodes = 10 # NOTE: we chose this
    total_reward_dummy = np.array([])

    for i in range(N_episodes):
        obs = env.reset()
        done = False
        
        #obs, reward, done = env.sample_action()
        
        while not done:
            #print(env.I)
            action = min_max_action(env, z_min, z_max)

            # Take the defined action (place an order), and advance to the next time period by taking a "step"
            obs, reward, done, _ = env.step(action)
            total_reward_dummy = np.append(total_reward_dummy, [reward])
            
    avg_total_rewards = np.average(total_reward_dummy)
    
    return avg_total_rewards


n_restarts = 100
best_min_max = -99999

for i in range(n_restarts):    
    z_up = np.array([random.randint(0, 10) for i in range(3)])
    z_low = np.array([random.randint(0, z_up[int(i)]) for i in range(len(z_up))])
    x0 = np.concatenate((z_low, z_up))
    
    opt_min_max = minimize(min_max_black_box, x0)
    #print(opt_base_stock.fun)
    
    if opt_min_max.fun > best_min_max:
        best_min_max = opt_min_max.fun

best_min_max

### Optimal (R, S) Policy

In [ ]:
def R_S_black_box(inputs):
    R_period = inputs[0]
    S = inputs[1:]
    
    env = InvManagementMasterEnv(periods=30, c=[4, 3, 2], I0=[3, 2, 2], dist_param={'mu': 1.2}, L=[1, 1, 1])

    N_episodes = 10 # NOTE: we chose this
    total_reward_dummy = np.array([])

    for i in range(N_episodes):
        obs = env.reset()
        done = False
        
        #obs, reward, done = env.sample_action()
        
        while not done:
            #print(env.I)
            action = R_S_action(env, R_period, S)

            # Take the defined action (place an order), and advance to the next time period by taking a "step"
            obs, reward, done, _ = env.step(action)
            total_reward_dummy = np.append(total_reward_dummy, [reward])
            
    avg_total_rewards = np.average(total_reward_dummy)
    
    return avg_total_rewards


n_restarts = 100
best_R_S = -99999

for i in range(n_restarts):    
    R_guess = np.array([random.randint(1, 5)])
    S_guess = np.array([random.randint(0, 10) for i in range(3)])
    x0 = np.concatenate((R_guess, S_guess))
    
    opt_R_S = minimize(R_S_black_box, x0)
    #print(opt_base_stock.fun)
    
    if opt_R_S.fun > best_R_S:
        best_R_S = opt_R_S.fun

best_R_S

### Optimal (R, s, S) Policy

In [ ]:
def R_s_S_black_box(inputs):
    R_period = inputs[0]
    s = inputs[1:-3]
    S = inputs[-3:]
    
    env = InvManagementMasterEnv(periods=30, c=[4, 3, 2], I0=[3, 2, 2], dist_param={'mu': 1.2}, L=[1, 1, 1])

    N_episodes = 10 # NOTE: we chose this
    total_reward_dummy = np.array([])

    for i in range(N_episodes):
        obs = env.reset()
        done = False
        
        #obs, reward, done = env.sample_action()
        
        while not done:
            #print(env.I)
            action = R_s_S_action(env, R_period, s, S)

            # Take the defined action (place an order), and advance to the next time period by taking a "step"
            obs, reward, done, _ = env.step(action)
            total_reward_dummy = np.append(total_reward_dummy, [reward])
            
    avg_total_rewards = np.average(total_reward_dummy)
    
    return avg_total_rewards


n_restarts = 100
best_R_s_S = -99999

for i in range(n_restarts):    
    R_guess = np.array([random.randint(1, 5)])
    S_guess = np.array([random.randint(0, 10) for i in range(3)])
    s_guess = np.array([random.randint(0, z_up[int(i)]) for i in range(len(z_up))])
    x0 = np.concatenate((R_guess, s_guess, S_guess))
    
    opt_R_s_S = minimize(R_s_S_black_box, x0)
    #print(opt_base_stock.fun)
    
    if opt_R_s_S.fun > best_R_s_S:
        best_R_s_S = opt_R_s_S.fun

best_R_s_S

### Optimal (r, Q) Policy

In [ ]:
def r_Q_black_box(inputs):
    r = inputs[:int(len(inputs)/2)]
    Q = inputs[int(len(inputs)/2):]
    
    env = InvManagementMasterEnv(periods=30, c=[4, 3, 2], I0=[3, 2, 2], dist_param={'mu': 1.2}, L=[1, 1, 1])

    N_episodes = 10 # NOTE: we chose this
    total_reward_dummy = np.array([])

    for i in range(N_episodes):
        obs = env.reset()
        done = False
        
        #obs, reward, done = env.sample_action()
        
        while not done:
            #print(env.I)
            action = r_Q_action(env, r, Q)

            # Take the defined action (place an order), and advance to the next time period by taking a "step"
            obs, reward, done, _ = env.step(action)
            total_reward_dummy = np.append(total_reward_dummy, [reward])
            
    avg_total_rewards = np.average(total_reward_dummy)
    
    return avg_total_rewards


n_restarts = 100
best_r_Q = -99999

for i in range(n_restarts):    
    r_guess = np.array([random.randint(1, 5) for i in range(3)])
    Q_guess = np.array([random.randint(0, 10) for i in range(3)])
    x0 = np.concatenate((r_guess, Q_guess))
    
    opt_r_Q = minimize(r_Q_black_box, x0)
    #print(opt_base_stock.fun)
    
    if opt_r_Q.fun > best_r_Q:
        best_r_Q = opt_r_Q.fun

best_r_Q

### Random agent

In [ ]:
env = InvManagementMasterEnv(periods=30, c=[4, 3, 2], I0=[3, 2, 2], dist_param={'mu': 1.2}, L=[1, 1, 1])

N_episodes = 10 # NOTE: we chose this
total_reward_dummy = np.array([])

for i in range(N_episodes):
    obs = env.reset()
    done = False
    
    #obs, reward, done = env.sample_action()
    
    while not done:
        #print(env.I)
        action = env.sample_action()

        # Take the defined action (place an order), and advance to the next time period by taking a "step"
        obs, reward, done, _ = env.step(action)
        total_reward_dummy = np.append(total_reward_dummy, [reward])
        
avg_total_rewards = np.average(total_reward_dummy)
avg_total_rewards